# Construct Main Dataset of ED stays, linked with additional info from MIMIC-IV-ED

Note:  MIMIC-ED should be downloaded at “data/ed" folder (please see the README for more instructions).  

Original draft of this pre-processing code from https://github.com/nliulab/mimic4ed-benchmark.  


## Prepare Python library and raw data

In [5]:
# reload packages from notebook whenever needed
%load_ext autoreload
%autoreload 2
import os

# Add /home/rm868/granular-race-disparities_MLHC23/preprocessing to path so that local imports work
import sys
sys.path.append('/home/rm868/granular-race-disparities_MLHC23/preprocessing')

from helpers import *
from medcode_utils import comorbidity, extract_icd_list
from paths import mimic_iv_path, data_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Define auxillary paths/files
# 'ed' subfolder should also be added to mimic_iv_path
mimic_iv_core_path = os.path.join(mimic_iv_path, 'hosp')
mimic_iv_hosp_path = os.path.join(mimic_iv_path , 'hosp')   
mimic_iv_icu_path = os.path.join(mimic_iv_path, 'icu')
mimic_iv_ed_path = os.path.join(mimic_iv_path, 'ed')

# Use gzipped filepaths for auxiliary MIMIC data
icu_filename_dict = {"chartevents":"chartevents.csv.gz", "datetimeevents":"datetimeevents.csv.gz","d_items":"d_items.csv.gz","icustays":"icustays.csv.gz","inputevents":"inputevents.csv.gz","outputevents":"outputevents.csv.gz","procedureevents":"procedureevents.csv.gz"}
core_filename_dict = {"patients":"patients.csv.gz", "admissions":"admissions.csv.gz", "transfers":"transfers.csv.gz"}
hosp_filename_dict = {"d_hcpcs":"d_hcpcs.csv.gz","d_icd_diagnoses":"d_icd_diagnoses.csv.gz","d_labitems":"d_labitems.csv.gz","emar":"emar.csv.gz","hcpcsevents":"hcpcsevents.csv.gz","microbiologyevents":"microbiologyevents.csv.gz","poe":"poe.csv.gz","prescriptions":"prescriptions.csv.gz","services":"services.csv.gz","diagnoses_icd":"diagnoses_icd.csv.gz","d_icd_procedures":"d_icd_procedures.csv.gz","drgcodes":"drgcodes.csv.gz","emar_detail":"emar_detail.csv.gz","labevents":"labevents.csv.gz","pharmacy":"pharmacy.csv.gz","poe_detail":"poe_detail.csv.gz","procedures_icd":"procedures_icd.csv.gz"}
ed_filename_dict = {'diagnosis':'diagnosis.csv.gz', 'edstays':'edstays.csv.gz',  'medrecon':'medrecon.csv.gz',  'pyxis':'pyxis.csv.gz',  'triage':'triage.csv.gz',  'vitalsign':'vitalsign.csv.gz'}

complaint_dict = {"chiefcom_chest_pain" : "chest pain", "chiefcom_abdominal_pain" : "abdominal pain|abd pain", 
"chiefcom_headache" : "headache|lightheaded", "chiefcom_shortness_of_breath" : "breath", "chiefcom_back_pain" : "back pain", "chiefcom_cough" : "cough", 
"chiefcom_nausea_vomiting" : "nausea|vomit", "chiefcom_fever_chills" : "fever|chill", "chiefcom_syncope" :"syncope", "chiefcom_dizziness" : "dizz"}

# Defining parameters for the outcome variables
icu_transfer_timerange = 12 # Considered critical outcome if patient is transferred to ICU within 12 hours of ED visit
next_ed_visit_timerange = 3 # Considered an ED revisit if patient returns to ED within 3 days of discharge


## Load raw data tables through pandas library

In [7]:
## Reading main tables
df_edstays = read_edstays_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['edstays']))
if os.path.exists(mimic_iv_core_path):
    df_patients = read_patients_table(os.path.join(mimic_iv_core_path, core_filename_dict['patients']))
    df_admissions = read_admissions_table(os.path.join(mimic_iv_core_path, core_filename_dict["admissions"]))
else:
    df_patients = read_patients_table(os.path.join(mimic_iv_hosp_path, core_filename_dict['patients']))
    df_admissions = read_admissions_table(os.path.join(mimic_iv_hosp_path, core_filename_dict["admissions"]))
df_icustays = read_icustays_table(os.path.join(mimic_iv_icu_path, icu_filename_dict['icustays']))
df_triage = read_triage_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['triage']))
df_vitalsign = read_vitalsign_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['vitalsign']))
df_pyxis = read_pyxis_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['pyxis']))
df_medrecon = read_pyxis_table(os.path.join(mimic_iv_ed_path, ed_filename_dict['medrecon']))

## Read data here for ICD
df_diagnoses = read_diagnoses_table(os.path.join(mimic_iv_hosp_path, hosp_filename_dict['diagnoses_icd']))


## ED root table, demographic and outcomes

In [8]:
## Merging patients -> merging admissions -> merging triage -> master
df_main = merge_edstays_patients_on_subject(df_edstays, df_patients)
df_main = merge_edstays_admissions_on_subject(df_main, df_admissions)

In [9]:
## Adding age, mortality and ICU transfer outcome
df_main = add_age(df_main)
df_main = add_inhospital_mortality(df_main)
df_main = add_ed_los(df_main)
df_main = add_outcome_icu_transfer(df_main, df_icustays, icu_transfer_timerange)
df_main['outcome_hospitalization'] = ~pd.isnull(df_main['hadm_id'])
df_main['outcome_critical'] = df_main['outcome_inhospital_mortality'] | df_main[''.join(['outcome_icu_transfer_', str(icu_transfer_timerange), 'h'])]

# Sort main table for further processing
df_main = df_main.sort_values(['subject_id', 'intime']).reset_index()

# Filling subjects NA ethnicity, takes ~17s
df_main = fill_na_ethnicity(df_main)

## Health Utilization

In [14]:
# Generate past ED visits
df_main = generate_past_ed_visits(df_main, timerange=30)
df_main = generate_past_ed_visits(df_main, timerange=90)
df_main = generate_past_ed_visits(df_main, timerange=365)

In [15]:
# Outcome:  future ED revisit variables
df_main = generate_future_ed_visits(df_main, next_ed_visit_timerange)

In [16]:
# Generate past admissions
df_main = generate_past_admissions(df_main, df_admissions, timerange=30)
df_main = generate_past_admissions(df_main, df_admissions, timerange=90)
df_main = generate_past_admissions(df_main, df_admissions, timerange=365)

/home/rm868/granular-race-disparities_MLHC23/preprocessing/helpers.py:209: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_admissions_sorted.loc[:,'admittime'] = pd.to_datetime(df_admissions_sorted['admittime'])


/home/rm868/granular-race-disparities_MLHC23/preprocessing/helpers.py:209: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_admissions_sorted.loc[:,'admittime'] = pd.to_datetime(df_admissions_sorted['admittime'])


/home/rm868/granular-race-disparities_MLHC23/preprocessing/helpers.py:209: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_admissions_sorted.loc[:,'admittime'] = pd.to_datetime(df_admissions_sorted['admittime'])


In [17]:
# Generate past icu visits
df_main  = generate_past_icu_visits(df_main, df_icustays, timerange=30)
df_main  = generate_past_icu_visits(df_main, df_icustays, timerange=90)
df_main  = generate_past_icu_visits(df_main, df_icustays, timerange=365)

In [18]:
# Generate numeric timedelta variables
df_main = generate_numeric_timedelta(df_main)

## Triage Information

In [20]:
# Merging with triage table, cols of form triage_*
df_main = merge_edstays_triage_on_subject(df_main, df_triage)

In [21]:
# Encoding 10 chief complaints, chiefcom_*
df_main = encode_chief_complaints(df_main, complaint_dict)

## Comorbidities from diagnosis ICD

In [24]:
# This function takes about 40 min
df_main = comorbidity(df_main, df_diagnoses, df_admissions, timerange = 365*5)
extract_icd_list(df_edstays, df_diagnoses, df_admissions, data_path, timerange = 365*5, version = 'v9')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, data_path, timerange = 365*5, version = 'v9_3digit')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, data_path, timerange = 365*5, version = 'v10')


/home/rm868/granular-race-disparities_MLHC23/preprocessing/medcode_utils.py:94: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_diagnoses_with_adm.loc[:, 'dischtime'] = pd.to_datetime(


/home/rm868/granular-race-disparities_MLHC23/preprocessing/medcode_utils.py:94: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_diagnoses_with_adm.loc[:, 'dischtime'] = pd.to_datetime(


Number of unique ICD codes v9:  8664


/home/rm868/granular-race-disparities_MLHC23/preprocessing/medcode_utils.py:94: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_diagnoses_with_adm.loc[:, 'dischtime'] = pd.to_datetime(


Number of unique ICD codes v9_3digit:  8609


/home/rm868/granular-race-disparities_MLHC23/preprocessing/medcode_utils.py:94: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_diagnoses_with_adm.loc[:, 'dischtime'] = pd.to_datetime(


Number of unique ICD codes v10:  14956


## ED vital signs

In [25]:
df_main = merge_vitalsign_info_on_edstay(df_main, df_vitalsign, options=['last'])

## Medication

In [26]:
# Medication counts
# See more info about medrecon/pyxis here: https://physionet.org/content/mimic-iv-ed/2.2/
df_main = merge_med_count_on_edstay(df_main, df_pyxis)
df_main = merge_medrecon_count_on_edstay(df_main, df_medrecon)

## Remove outliers
They will be imputed later, at training time

In [30]:
from helpers import vitals_valid_range

df_main = convert_temp_to_celsius(df_main)
display_outliers_count(df_main, vitals_valid_range)

,variable,< outlier_low,"[outlier_low, valid_low)","[valid_low, valid_high]","(valid_high, outlier_high]",> outlier_high
0,triage_temperature,479,29,401155,0,9
1,triage_heartrate,0,0,407990,0,7
2,triage_resprate,0,0,404731,0,3
3,triage_o2sat,0,0,404444,6,41
4,triage_sbp,0,0,406780,0,16
5,triage_dbp,0,0,405601,0,395
6,triage_pain,0,0,386851,0,11205
7,triage_acuity,0,0,418100,0,0
8,ed_temperature_last,751,43,398206,1,48
9,ed_heartrate_last,0,0,406649,0,0


In [31]:
df_main = remove_outliers(df_main, vitals_valid_range)

## Add clinical risk scores

In [33]:
add_triage_MAP(df_main)
add_score_CCI(df_main)
add_score_CART(df_main)
add_score_NEWS(df_main)

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_NEWS' successfully added


## Filter

Ensure that age >= 18 and triage_acuity is a present variable.

In [35]:
print('Before filtering for "age" >= 18 : master dataset size = ', len(df_main))
df_main = df_main[df_main['age'] >= 18]
print('After filtering for "age" >= 18 : master dataset size = ', len(df_main))

print('Before filtering for non-null "triage_acuity": master dataset size = ', len(df_main))
df_main = df_main[df_main['triage_acuity'].notnull()]
print('After filtering for non-null "triage_acuity": master dataset size = ', len(df_main))

Before filtering for "age" >= 18 : master dataset size =  425087
After filtering for "age" >= 18 : master dataset size =  425087
Before filtering for non-null "triage_acuity" >= 18 : master dataset size =  425087
After filtering for non-null "triage_acuity" >= 18 : master dataset size =  418100


## Create granular and coarse race columns

In [40]:
granular_to_coarse = {
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC OR LATINO', 
    'HISPANIC/LATINO - CUBAN': 'HISPANIC OR LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC OR LATINO',
    'SOUTH AMERICAN': 'HISPANIC OR LATINO',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - KOREAN': 'ASIAN',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - RUSSIAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'PORTUGUESE': 'WHITE',
    'BLACK/CAPE VERDEAN': 'BLACK/AFRICAN AMERICAN',
    'BLACK/CARIBBEAN ISLAND': 'BLACK/AFRICAN AMERICAN',
    'BLACK/AFRICAN': 'BLACK/AFRICAN AMERICAN',
    'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER',
    'MULTIPLE RACE/ETHNICITY': 'OTHER',
    'UNKNOWN': 'OTHER',
    'PATIENT DECLINED TO ANSWER': 'OTHER',
    'UNABLE TO OBTAIN': 'OTHER',
}

'''
Input: patient reported race recorded in MIMIC-IV
Output: 
If the reported race is granular, then return its coarse category;
else, return the reported (coarse) race.

The goal is to construct a coarse race column for all patients.
'''
def coarsen_race(race):
    if race in granular_to_coarse: return granular_to_coarse[race]
    else: return race

In [50]:
df_main.insert(7, 'coarse_race', df_main['race'].apply(coarsen_race))
df_main.rename(columns={'race': 'granular_race'}, inplace=True)

print(df_main['coarse_race'].value_counts())
print()

print("Number of visits with a granular race reported")
print((df_main.granular_race != df_main.coarse_race).sum())

print("Number of visits with only a coarse race reported")
print((df_main.granular_race == df_main.coarse_race).sum())

WHITE                     244093
BLACK/AFRICAN AMERICAN     92168
HISPANIC OR LATINO         35205
OTHER                      28313
ASIAN                      18321
Name: coarse_race, dtype: int64

Number of visits with a granular race reported
86371
Number of visits with only a coarse race reported
331729


## Review the main dataset and output

In [51]:
df_main.to_csv(os.path.join(data_path, 'main_ed_dataset.csv'), index=False)

In [52]:
df_main